In [36]:
import pandas as pd

Make sure I've not mucked this up with ordering things! Every doc should have a topic

In [37]:
HeatData=pd.read_csv('DocumentProcessing/HeatData.csv')
HeatData.head()

ColdData=pd.read_csv('DocumentProcessing/ColdData.csv')
ColdData.head()

FloodData=pd.read_csv('DocumentProcessing/FloodData.csv')
FloodData.head()

,Document,Index
0,I moved my outdoor furniture,1
1,was anxious when I heard the wind,1
2,No dog walks,2
3,Our garden flooded so had to fork it to aid dr...,3
4,Didn’t go out when storms hit,4


In [38]:
ColdTopics = pd.read_excel('DocumentProcessing/Cold_Document_Info (1).xlsx')
ColdTopics['Topic_Actual'] = ColdTopics.apply(
    lambda row: int(row['Reassigned Topic']) if pd.notnull(row['Reassigned Topic']) else int(row['Topic']), axis=1
)
ColdFocus = ColdTopics[["Document", "Topic_Actual"]]#, 'Representation', 'Representative_Docs', 'Top_n_words']]
ColdFocus.head()

FloodTopics = pd.read_excel('DocumentProcessing/Flood_Document_Info (1).xlsx')
FloodTopics['Topic_Actual'] = FloodTopics.apply(
    lambda row: int(row['Reassigned Topic']) if pd.notnull(row['Reassigned Topic']) else int(row['Topic']), axis=1
)
FloodFocus = FloodTopics[["Document", "Topic_Actual"]]#, 'Representation', 'Representative_Docs', 'Top_n_words']]
FloodFocus.head()

,Document,Topic_Actual
0,I moved my outdoor furniture,0
1,was anxious when I heard the wind,1
2,No dog walks,6
3,Our garden flooded so had to fork it to aid dr...,2
4,Didnâ€™t go out when storms hit,1


In [39]:
# Remove any entry with "Topic_Actual" == -100
print("Cold entries removed: ", ColdFocus[ColdFocus['Topic_Actual'] == -100].shape[0])
ColdFocus = ColdFocus[ColdFocus['Topic_Actual'] != -100]
# Output how many ColdFocus items were removed by this process
print("Flood entries removed: ", FloodFocus[FloodFocus['Topic_Actual'] == -100].shape[0])
FloodFocus = FloodFocus[FloodFocus['Topic_Actual'] != -100]

Cold entries removed:  126
Flood entries removed:  107


In [40]:
# Note I reordered the relabelled data by index in excel before merging
ColdDf = ColdData.merge(ColdFocus)
ColdDf.head()

FloodDf = FloodData.merge(FloodFocus)
print(FloodDf)

                                               Document  Index  Topic_Actual
0                         I moved my outdoor furniture       1             0
1                    was anxious when I heard the wind       1             1
2                                         No dog walks       2             6
3     Our garden flooded so had to fork it to aid dr...      3             2
4                                       Prepared for it      5            11
...                                                 ...    ...           ...
2498                                     Stayed in more   1320             4
2499                                     Stayed in more   1320             4
2500                                     Stayed in more   1320             4
2501                                     Stayed in more   1320             4
2502                                  Walked less often   1321            15

[2503 rows x 3 columns]


In [41]:
ColdDf["Topic_Actual"].value_counts()
FloodDf["Topic_Actual"].value_counts()

Topic_Actual
 4     674
 0     272
 1     176
 2     151
 10    132
 5     130
 3     125
 9     105
 8     100
 7      99
-1      95
 6      87
 13     83
 14     63
 11     60
 12     57
 15     50
 16     44
Name: count, dtype: int64

In [42]:
def ColdMap(topic_no):
    """
    Maps topic numbers generated from BERT for Cold Data to major and minor categories
    """
    if topic_no == 0:
        return "Personal care", "Warm clothes/hot water bottle"
    elif topic_no == 1:
        return "Reduced activity", "Cancelled plans/exercise"
    elif topic_no == 2:
        return "Precaution", "Car/external home preparation"
    elif topic_no == 3:
        return "Home adaptation", "Blankets"
    elif topic_no in [4, 7, 8, 9, 14]:
        return "Technology solution", "Heating"
    elif topic_no == 5:
        return "Alternative plans", "Transport"
    elif topic_no == 6:
        return "Reduced activity", "Stay indoors"
    elif topic_no == 10:
        return "Alternative plans", "Early bed/walk"
    elif topic_no in [11, 15]:
        return "Home adaptation", "Insulation"
    elif topic_no == 12:
        return "Alternative plans", "Home working"
    elif topic_no == 13:
        return "Anxiety", "Cost of heating"
    elif topic_no == 16:
        return "Precaution", "Stock up food"
    elif topic_no == 17:
        return "Personal care", "Hot meals/drinks"
    elif topic_no == -1:
        return "Other", "Other"
    else:
        print("Cold Error:", topic_no)
    
def FloodMap(topic_no):
    """
    Maps topic numbers generated from BERT for Flood Data to major and minor categories
    """
    if topic_no == 0:
        return "Precaution", "Secure objects"
    elif topic_no in [1, 4, 5, 10]:
        return "Reduced activity", "Stay indoors"
    elif topic_no in [2, 14]:
        return "Alternative plans", "Route"
    elif topic_no in [3, 15]:
        return "Reduced activity", "Reduced exercise"
    elif topic_no == 6:
        return "Clothing", "Waterproof gear"
    elif topic_no == 7:
        return "Alternative plans", "Transport"
    elif topic_no == 8:
        return "Alternative plans", "Longer journeys"
    elif topic_no == 9:
        return "Reduced activity", "Cancelled plans"
    elif topic_no == 11:
        return "Precaution", "Stock up food"
    elif topic_no == 12:
        return "Clothing", "Warm clothes"
    elif topic_no == 13:
        return "Precaution", "Clear drains/gutter"
    elif topic_no == 16:
        return "Precaution", "Check in on people"
    elif topic_no == -1:
        return "Other", "Other"
    else:
        print("Flood Error", topic_no)

In [43]:
ColdDf['Major'], ColdDf['Minor'] = zip(*ColdDf['Topic_Actual'].map(ColdMap))
ColdDf.head()

FloodDf['Major'], FloodDf['Minor'] = zip(*FloodDf['Topic_Actual'].map(FloodMap))
FloodDf

,Document,Index,Topic_Actual,Major,Minor
0,I moved my outdoor furniture,1,0,Precaution,Secure objects
1,was anxious when I heard the wind,1,1,Reduced activity,Stay indoors
2,No dog walks,2,6,Clothing,Waterproof gear
3,Our garden flooded so had to fork it to aid dr...,3,2,Alternative plans,Route
4,Prepared for it,5,11,Precaution,Stock up food
...,...,...,...,...,...
2498,Stayed in more,1320,4,Reduced activity,Stay indoors
2499,Stayed in more,1320,4,Reduced activity,Stay indoors
2500,Stayed in more,1320,4,Reduced activity,Stay indoors
2501,Stayed in more,1320,4,Reduced activity,Stay indoors


https://plotly.com/python/icicle-charts/

In [44]:
# cold_major_count = ColdDf['Major'].value_counts()
# print(cold_major_count)
# cold_minor_count = ColdDf['Minor'].value_counts()
# print(cold_minor_count)

def MajorMinorDf_To_IcicleFrame(dataframe):
    """
    Converts a dataframe with Major and Minor columns to a dataframe with Major, Minor, and Count columns.
    The Count column represents the unique number of individuals (Index) that correspond with each combination
    of Major and Minor.
    """
    
    # Group by 'Major' and 'Minor' and count unique occurrences of 'Index'
    major_minor_count = dataframe.groupby(['Major', 'Minor'])['Index'].nunique().reset_index(name='Count')
    
    # Sort the DataFrame by 'Major' and 'Count' in descending order
    major_minor_count = major_minor_count.sort_values(by=['Major', 'Count'], ascending=[True, False])
    
    # Reset the index of the sorted DataFrame
    major_minor_count = major_minor_count.reset_index(drop=True)
    
    # Return the processed DataFrame
    return major_minor_count

Cold_Icicle = MajorMinorDf_To_IcicleFrame(ColdDf)
Flood_Icicle = MajorMinorDf_To_IcicleFrame(FloodDf)

In [45]:
Cold_Icicle = Cold_Icicle[Cold_Icicle['Major'] != 'Anxiety']
Cold_Icicle.reset_index(drop=True, inplace=True)
Cold_Icicle

,Major,Minor,Count
0,Alternative plans,Transport,110
1,Alternative plans,Early bed/walk,92
2,Alternative plans,Home working,80
3,Home adaptation,Blankets,162
4,Home adaptation,Insulation,126
5,Other,Other,52
6,Personal care,Warm clothes/hot water bottle,561
7,Personal care,Hot meals/drinks,71
8,Precaution,Car/external home preparation,213
9,Precaution,Stock up food,67


https://plotly.com/python/discrete-color/

In [85]:
import plotly.express as px

fig = px.colors.qualitative.swatches()
fig.show()

In [88]:
import plotly.express as px

fig = px.icicle(Cold_Icicle, path=[px.Constant("all"), 'Major', 'Minor'], values='Count', 
                color_discrete_sequence=px.colors.qualitative.Light24)
# Colour options (see above)
fig.update_traces(root_color="lightgrey", textinfo='label+value', texttemplate='%{label}: %{value}')
fig.update_layout(title="Unique Respondents for different Topics on Cold Adaptations", font_size=20, title_font_size=23)
# font_size controls the global font size and it will make things fit where needed
# title_font_size controls the title font size which we want to be different from the global font_size
fig.update_layout(width=800, height=1000)
fig.show()

In [87]:
import plotly.express as px

fig = px.icicle(Flood_Icicle, path=[px.Constant("all"), 'Major', 'Minor'], values='Count', 
                color_discrete_sequence=px.colors.qualitative.Light24)
# Colour options (see above)
fig.update_traces(root_color="lightgrey", textinfo='label+value', texttemplate='%{label}: %{value}')
fig.update_layout(title="Unique Respondents for different Topics on Flood Adaptations", font_size=20, title_font_size=23)
# font_size controls the global font size and it will make things fit where needed
# title_font_size controls the title font size which we want to be different from the global font_size
fig.update_layout(width=800, height=1000)
fig.show()